# Лабораторная работа 1 "Моделирование базовых случайных величин (БСВ)"

## Ширяева Ольга, 4 курс, 3 группа

## Вариант 12

## Задание 1.

## Осуществить моделирование n = 1000 реализаций БСВ с помощь мультипликативного конгруэнтного метода (МКМ) с параметрами a 0 =  β = 161 051, M = 2 ^ 31

In [1]:
alpha = 161051
beta = alpha
M = 2 ** 31
n = 1000

In [2]:
# multiplicative congruential generator (MCG)
def mcg(alpha, beta, M, n):
    for i in range(n):
        alpha = (beta * alpha) % M
        yield alpha / M    

In [3]:
mcg_seq = list(mcg(alpha, beta, M, n))

for i in range(10):  
    print(mcg_seq[i])

0.07805452914908528
0.7599739893339574
0.5709562231786549
0.07069914555177093
0.16809025825932622
0.10418292274698615
0.7638913248665631
0.4617610848508775
0.08447631867602468
0.9955990924499929


## Задание 2. 
## Осуществить моделирование n = 1000 реализаций БСВ с помощью метода Макларена-Марсальи (один датчик должен быть мультипликативно конгруэнтный (п. 1), второй – на выбор).  K = 192 – объем вспомогательной таблицы.

In [17]:
K = 192
mcg_list = list(mcg(alpha, beta, M, n + K))
random_list = list(mcg(alpha + 2 * K, beta + 2 * K, 2 * M, n + K))

In [18]:
# MacLaren-Marsaglia generator (MMG)
def mmg(rand1, rand2, k, n):
    result = [None] * (n)
    result[:k] = rand1[:k]
    for i in range(n):
        s = int(rand2[i] * k)
        yield result[s]
        result[s] = rand1[i + k]        

In [19]:
mmg_seq = list(mmg(mcg_list, random_list, K, n)) 

for i in range(10):  
    print(mmg_seq[i])

0.9623539834283292
0.16787964524701238
0.15060700615867972
0.189916857983917
0.69866788899526
0.7865513390861452
0.6450425540097058
0.4766780664213002
0.5709562231786549
0.24340255977585912


## Проверить точность моделирования обоих датчиков (п. 1 и п. 2) с помощью критерия согласия Колмогорова и chi-2-критерия Пирсона с уровнем значимости ε = 0.05.

In [20]:
def sort_and_count(n, rand_seq):
    seq = sorted(rand_seq, key=float)
    size = len(seq)
    segment_size = int(size / n)
    probability = 1 / n
    frequencies = []
    j = 0
    for i in range(n):
        count = 0    
        while j < size and seq[j] < (i + 1) / n:
                j += 1
                count += 1
        frequencies.append(count)
    return frequencies, segment_size, probability


def chi_square(frequencies, size, probability, delta):
    chi = 0
    for frequency in frequencies:
        chi += (frequency - probability * size) ** 2 / (probability * size)
    print('chi-square : {}, delta : {}'.format(chi, delta))
    
    return chi < delta

###  Квантили распределения хи-квадрат 
### https://ru.wikipedia.org/wiki/%D0%9A%D0%B2%D0%B0%D0%BD%D1%82%D0%B8%D0%BB%D0%B8_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D1%85%D0%B8-%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82

In [21]:
delta_10_005 = 16.919
delta_40_005 = 54.572
epsilon = 0.05
k = 10

frequencies, segment_size, probability = sort_and_count(k, mmg_seq)

print('frequencies : {}'.format(frequencies))
print('segment size : {}'.format(segment_size))
print('probability : {}'.format(probability))

chi_square(frequencies, len(mmg_seq), probability, delta_10_005)

frequencies : [111, 102, 104, 104, 97, 103, 110, 88, 95, 86]
segment size : 100
probability : 0.1
chi-square : 6.3999999999999995, delta : 16.919


True

In [22]:
frequencies, segment_size, probability = sort_and_count(k, mcg_seq)

print('frequencies : {}'.format(frequencies))
print('segment size : {}'.format(segment_size))
print('probability : {}'.format(probability))

chi_square(frequencies, len(mcg_seq), probability, delta_10_005)

frequencies : [110, 103, 104, 106, 98, 95, 119, 89, 95, 81]
segment size : 100
probability : 0.1
chi-square : 10.58, delta : 16.919


True

### Критерий согласия Колмогорова

### https://arhiuch.ru/lab11.html

In [10]:
import numpy as np


def kolmogorov(rand_seq):
    delta = 1.358
    
    n = len(rand_seq)
    empirical = sorted(rand_seq, key=float)
    theoretical = [float(i / n) for i in range(n)]
    difference = 0.
    
    for i, value in enumerate(empirical):
        difference = max(difference, abs(value - theoretical[i]))
       
    value = difference * np.sqrt(n)
    print('kolmogorov : {}, delta : {}'.format(value, delta))
    
    return value < delta    

In [11]:
kolmogorov(mcg_seq)

kolmogorov : 1.3324792908737209, delta : 1.358


True

In [12]:
kolmogorov(mmg_seq)

kolmogorov : 1.1743654078653019, delta : 1.358


True